In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import transformers

In [3]:
from dataset import JSONLDataset

tokenizer = transformers.GPT2Tokenizer.from_pretrained('gpt2')
tokenizer.pad_token = tokenizer.eos_token
tokenizer.model_max_length = 64
dataset = JSONLDataset('data/proj2_data.jsonl', tokenizer)

print(len(dataset))
print(dataset[60000])

1685162
(tensor([   49, 38165,     3,   318,  4795,   286,   720,    49, 35307,   198,
          198,  9690,   284, 20607,  2611,  3467,    58, 10671,  2611,    15,
           59,    60,  1978,   351, 35125,   356,   423,   326, 32382,    70,
            6,     7,    74,     8, 10779,    59, 30300,     7,    32, 23330,
           74,    92, 19415,   293,   532,    59,  9464,   685,    34, 36796,
           12,    16, 32239,    11,    70,     7,    74, 19415,  3506,    60,
           61, 31478, 31944,   352]), tensor([38165,     3,   318,  4795,   286,   720,    49, 35307,   198,   198,
         9690,   284, 20607,  2611,  3467,    58, 10671,  2611,    15,    59,
           60,  1978,   351, 35125,   356,   423,   326, 32382,    70,     6,
            7,    74,     8, 10779,    59, 30300,     7,    32, 23330,    74,
           92, 19415,   293,   532,    59,  9464,   685,    34, 36796,    12,
           16, 32239,    11,    70,     7,    74, 19415,  3506,    60,    61,
        31478, 31

In [6]:
from mingpt.model import GPT
from mingpt.trainer import Trainer

model_config = GPT.get_default_config()
model_config.model_type = "gpt-nano"
model_config.vocab_size = tokenizer.vocab_size
model_config.block_size = tokenizer.model_max_length
model = GPT(model_config)

trainer_config = Trainer.get_default_config()
trainer_config.max_iters = 10
trainer_config.device = "mps"
trainer_config.load_checkpoint = True
trainer_config.checkpoint_dir = "checkpoints"
trainer_config.checkpoint_iters = 1
trainer = Trainer(trainer_config, model, dataset)

def print_loss(trainer):
    if trainer.iter_num % 2 == 0:
        print(f"Batch: {trainer.iter_num}, Loss: {trainer.loss.item()}")

trainer.add_callback("on_batch_end", print_loss)

number of parameters: 2.50M
running on device mps


In [8]:
trainer.run()

Batch: 0, Loss: 10.463829040527344
Batch: 2, Loss: 10.388639450073242
Batch: 4, Loss: 10.387161254882812
Batch: 6, Loss: 10.351572036743164
Batch: 8, Loss: 10.331005096435547
